## **Bước 0. Đăng nhập Google**

In [1]:
# Chạy ô này để đăng nhập google
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


## **Bước 1. Nhập đường dẫn đến thư mục chứa dữ liệu test**

**TEST_ROOT_DIR** hợp lệ là một thư mục chứa các video cần dự đoán. Tương tự như cấu trúc thư mục **"Test - Vòng loại"** mà Ban tổ chức đã cung cấp.

<img alt="image" src="https://live.staticflickr.com/65535/53624122132_387b2c26d6_h.jpg">

Dán đường dẫn đến folder chứa dữ liệu test của BTC vào ô bên dưới và bấm chạy.


In [11]:
# Dán đường dẫn đến folder chứa dữ liệu test của bạn tại đây
TEST_ROOT_DIR = '/content/drive/MyDrive/Dữ liệu vòng chung kết'

## **Bước 2. Cài đặt thư viện và tải mô hình**

In [3]:
!pip install decord seaborn transformers evaluate accelerate ffmpeg-python av -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 45.5 MB/s eta 0:00:00


In [4]:
!pip install --upgrade --no-cache-dir gdown -q

In [5]:
!gdown --folder https://drive.google.com/drive/folders/1aXQVoPqJHBlpDRuH64f9PlQlf2hFHdIU

Retrieving folder contents
Processing file 1GaXE1ZVV_qbf_YlQ3bAgPOtXBiv1Dt2n config.json
Processing file 1ws8NOiMx8pMAkIn-brDZOWp58lXWnB8k model.safetensors
Processing file 12TYEdvV0uYEBcfWEPGLGrnXlYI0d2FSK optimizer.pt
Processing file 1qEGvfrMWR9v5tNMzlT7jcmTJEDWbrvi_ preprocessor_config.json
Processing file 1JZpk-wKaN8FIIyw520TSUjFq5tigfbDo rng_state.pth
Processing file 1F36WH-XzGBohnAWDgnO6hGycsjc2pdmN scheduler.pt
Processing file 1tB9oAzR1DtVwUNxWPegtJG0Jh0eX0LdC trainer_state.json
Processing file 1VtPVJjiqrO0237r2udsNPPOElQXWMt_4 training_args.bin
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1GaXE1ZVV_qbf_YlQ3bAgPOtXBiv1Dt2n
To: /content/checkpoint-6000/config.json
100% 1.96k/1.96k [00:00<00:00, 10.5MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1ws8NOiMx8pMAkIn-brDZOWp58lXWnB8k
From (redirected): https://drive.google.com/uc?id=1ws8NOiMx8pMAkIn-brDZ

In [12]:
# Chạy ô này để kiểm tra mô hình đã tải xuống hay chưa
import os
CKPT = "/content/checkpoint-6000"
if os.path.exists(CKPT):
  print("Mô hình đã được tải xuống!")
else:
  print("Mô hình chưa được tải xuống")

Mô hình đã được tải xuống!


## **Bước 3. Chạy mô hình và in ra F1 score, confusion matrix và các thông số khác**

Chạy lần lượt hai ô lệnh dưới đây. Sau đó refresh panel ở bên trái sẽ thấy file csv chứa kết quả tên **kikiki-predict.csv**.

In [13]:
import os
import shutil
import uuid
import warnings
import math

import ffmpeg
import pandas as pd
import torch
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm
from transformers import (
    VideoMAEForVideoClassification,
    VideoMAEImageProcessor,
    pipeline,
)


ID2LABEL = {
    0: 'barbell biceps curl',
    1: 'bench press',
    2: 'chest fly machine',
    3: 'deadlift',
    4: 'decline bench press',
    5: 'hammer curl',
    6: 'hip thrust',
    7: 'incline bench press',
    8: 'lat pulldown',
    9: 'lateral raise',
    10: 'leg extension',
    11: 'leg raises',
    12: 'plank',
    13: 'pull Up',
    14: 'push-up',
    15: 'romanian deadlift',
    16: 'russian twist',
    17: 'shoulder press',
    18: 'squat',
    19: 't bar row',
    20: 'tricep Pushdown',
    21: 'tricep dips',
}


LABEL2ID = {
    'barbell biceps curl': 0,
    'bench press': 1,
    'chest fly machine': 2,
    'deadlift': 3,
    'decline bench press': 4,
    'hammer curl': 5,
    'hip thrust': 6,
    'incline bench press': 7,
    'lat pulldown': 8,
    'lateral raise': 9,
    'leg extension': 10,
    'leg raises': 11,
    'plank': 12,
    'pull Up': 13,
    'push-up': 14,
    'romanian deadlift': 15,
    'russian twist': 16,
    'shoulder press': 17,
    'squat': 18,
    't bar row': 19,
    'tricep Pushdown': 20,
    'tricep dips': 21,
}


warnings.filterwarnings("ignore")


def segment_video(video_length, chunk_size=10, overlap=3):
    if video_length < chunk_size + overlap:
        return [(0, video_length)]

    num_chunks = math.ceil((video_length - overlap) / (chunk_size - overlap))
    chunk_start = 0
    segments = []

    for i in range(num_chunks):
        chunk_end = min(chunk_start + chunk_size, video_length)
        segments.append((chunk_start, chunk_end))
        chunk_start += chunk_size - overlap

    return segments


def crop_video(video_file, start_time, end_time, save_dir):
    output_filename = os.path.join(save_dir, f"{uuid.uuid4()}.mp4")
    output_filename = str(output_filename)

    video_length = int(float(ffmpeg.probe(video_file)["format"]["duration"]))
    if start_time == 0 and end_time == video_length:
        shutil.copy(video_file, output_filename)
        return output_filename

    (
        ffmpeg.input(video_file, ss=start_time, to=end_time)
        .output(output_filename)
        .run(quiet=True)
    )
    return output_filename


def infer_segment(model_ckpt, video_path, chunk_size=5, overlap=1):
    device = "cuda" if torch.cuda.is_available() else "cpu"

    image_processor = VideoMAEImageProcessor.from_pretrained(model_ckpt)
    model = VideoMAEForVideoClassification.from_pretrained(
        model_ckpt,
        label2id=LABEL2ID,
        id2label=ID2LABEL,
        ignore_mismatched_sizes=True,
    )

    video_cls = pipeline(
        model=model,
        task='video-classification',
        feature_extractor=image_processor,
        device=device,
    )

    video_length = int(float(ffmpeg.probe(video_path)["format"]["duration"]))
    video_segments = segment_video(video_length, chunk_size, overlap)

    list_of_clips = []
    for start_time, end_time in video_segments:
        clip_path = crop_video(video_path, start_time, end_time, CROPPED_DIR)
        list_of_clips.append(clip_path)

    all_clip_predictions = {}
    for clip in list_of_clips:
        prediction = video_cls(clip)
        for pred in prediction:
            all_clip_predictions[pred["label"]] = (
                all_clip_predictions.get(pred["label"], 0) + pred["score"]
            )

    final_prediction = max(all_clip_predictions, key=all_clip_predictions.get)
    return final_prediction

In [14]:
CROPPED_DIR = "/content/temp"
os.makedirs(CROPPED_DIR, exist_ok=True)

all_results = []
for video_file in tqdm(os.listdir(TEST_ROOT_DIR)):
    video_path = os.path.join(TEST_ROOT_DIR, video_file)
    try:
        predict = infer_segment(CKPT, video_path, chunk_size=5, overlap=1)
        all_results.append((video_file, predict))
    except Exception as e:
        print(f"Error processing {video_file}: {e}")
        all_results.append((video_file, "error"))
        continue

df = pd.DataFrame(all_results, columns=["video", "predict"])
df.to_csv("kikiki-predict.csv", index=False)

100%|██████████| 22/22 [13:32<00:00, 36.92s/it]
